### Imports

In [1]:
import pandas as pd 
import os
%matplotlib inline

from os import listdir, path
from utils import *
from tensorflow.keras.layers import Conv2D,Input,ZeroPadding2D,BatchNormalization,Flatten,Activation,Dense,MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential

2022-05-03 23:37:21.056904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pedrod33/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-03 23:37:21.056960: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Loading Images

In [2]:
image_dir="/home/pedrod33/Desktop/MRSI/ano1s2/AA/Brain_Tumor_Detection/archive/brain_tumor_dataset/"

### Making directory for augmented images

**A directory is formed using os.makedirs() function for augmented images(yes/ no). Note- custom directory is obtained in outputs folder.**

In [3]:
if not path.exists('output/kaggle/working/augmented-images'):
    os.makedirs('output/kaggle/working/augmented-images')
if not path.exists('output/kaggle/working/augmented-images/yes'):
    os.makedirs('output/kaggle/working/augmented-images/yes')
if not path.exists('output/kaggle/working/augmented-images/no'):
    os.makedirs('output/kaggle/working/augmented-images/no')

## Augmentation of images 
**About the data:
The dataset contains 2 folders: yes and no which contains 253 Brain MRI Images. The folder yes contains 155 Brain MRI Images that are tumorous andno contains 98 Brain MRI Images that are non-tumorous.**

In [4]:
IMG_SIZE = 224
X, y = read_images()
X = crop_images(X)
aug_X, aug_y = augment_data(X,y, IMG_SIZE=IMG_SIZE)
aug_X = resize_and_rescale(aug_X, IMG_SIZE)
X = resize_and_rescale(X, IMG_SIZE)
X_train, y_train, X_val, y_val, X_test, y_test = split_and_shuffle(aug_X, aug_y)

X_train = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_train])
X_val = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_val])
X_test = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X])

2022-05-03 23:37:24.984615: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-03 23:37:24.984658: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pedrod33): /proc/driver/nvidia/version does not exist
2022-05-03 23:37:24.985722: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/pedrod33/Desktop/MRSI/ano1s2/AA/Brain_Tumor_Detection/utils.py:104: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify

## Visualization of data

In [5]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))

number of training examples = 303
number of test examples = 253


In [6]:
def build_model(input_shape):
    #removed normalization from last 2 conv layers because values were becoming too similar resulting in overfit
    model = Sequential()
    model.add(Input(input_shape)) 
    model.add(ZeroPadding2D((2, 2))) 
    #Conv2D(32, (7, 7), strides = (1, 1))(X_input)
    model.add(Activation('relu')) 

    model.add(Conv2D(filters=36, kernel_size=4, input_shape=input_shape, strides = (1, 1)))
    model.add(BatchNormalization(axis = 3, name = 'bn0'))
    model.add(MaxPooling2D((4,4), strides = (4, 4),padding="valid"))
    model.add(Dropout(.2))
    
    model.add(Conv2D(filters=64, kernel_size=4, input_shape=input_shape, strides = (1, 1)))
    model.add(MaxPooling2D((4,4), strides = (4,4),padding="valid"))
    model.add(Dropout(.3))
    
    model.add(Conv2D(filters=128, kernel_size=4, input_shape=input_shape, strides = (1, 1)))
    model.add(MaxPooling2D((4,4), strides = (4,4),padding="valid"))

    model.add(Flatten())
    
    model.add(Dense(1, activation='sigmoid')) 
    '''
    model.add(Conv2D(filters=32,kernel_size=(3,3),activation="leaky_relu",kernel_initializer="he_uniform",padding="same",input_shape=(IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="valid"))
    model.add(Dropout(0.1))
    model.add(Conv2D(filters=64,kernel_size=(3,3),activation="leaky_relu",kernel_initializer="he_uniform",padding="same"))
    model.add(MaxPooling2D(pool_size=(4,4),strides=4, padding="valid"))
    model.add(Dropout(0.3))
    model.add(Conv2D(filters=128,kernel_size=(3,3),activation="leaky_relu",kernel_initializer="he_uniform",padding="same"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(units=128,activation="leaky_relu",kernel_initializer="he_uniform"))
    model.add(Dense(units=1,activation="sigmoid"))# compile the modelmodel2.compile(optimizer=”adam”,loss=”binary_crossentropy”,metrics=[“accuracy”])
    '''
    
    
    
    return model

In [7]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE,3)
model=build_model(IMG_SHAPE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 228, 228, 3)      0         
 2D)                                                             
                                                                 
 activation (Activation)     (None, 228, 228, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 225, 225, 36)      1764      
                                                                 
 bn0 (BatchNormalization)    (None, 225, 225, 36)      144       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 36)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 56, 56, 36)       

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x=X_train, y=y_train, batch_size=32, epochs=22, validation_data=(X_val, y_val))

Epoch 1/22
10/10 [==============================] - 25s 3s/step - loss: 1.8218 - accuracy: 0.5050 - val_loss: 0.5910 - val_accuracy: 0.6436
Epoch 2/22
10/10 [==============================] - 18s 2s/step - loss: 0.7975 - accuracy: 0.5941 - val_loss: 0.5929 - val_accuracy: 0.6436
Epoch 3/22
10/10 [==============================] - 19s 2s/step - loss: 0.6594 - accuracy: 0.6898 - val_loss: 0.5841 - val_accuracy: 0.6436
Epoch 4/22
10/10 [==============================] - 18s 2s/step - loss: 0.5381 - accuracy: 0.7558 - val_loss: 0.5743 - val_accuracy: 0.6733
Epoch 5/22
10/10 [==============================] - 19s 2s/step - loss: 0.4620 - accuracy: 0.7822 - val_loss: 0.5950 - val_accuracy: 0.8218
Epoch 6/22
10/10 [==============================] - 20s 2s/step - loss: 0.3797 - accuracy: 0.8416 - val_loss: 0.5403 - val_accuracy: 0.6634
Epoch 7/22
10/10 [==============================] - 22s 2s/step - loss: 0.3276 - accuracy: 0.8647 - val_loss: 0.5140 - val_accuracy: 0.6931
Epoch 8/22
10/10 [==

### Plotting of accuracy

### model.evaluate(X_test, y)

In [9]:
model.evaluate(X_test, y)

8/8 [==============================] - 4s 451ms/step - loss: 0.3261 - accuracy: 0.9249


[0.3260865807533264, 0.9249011874198914]